# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a short definition of its context, we can use a model like GTP3.5-Turbo to select which tables are necessary to create an SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [57]:
import pandas as pd

data = {
    'table': [
        'hospitals(hosp_id, name, phone, address, city, state, zip, doctors, nurses, medical_specialty, treatment_success_rate)',
        'patients(hosp_id,  patient_id, first_name, last_name, admission_date, symptoms, diagnosis_id, phone, email, address, treatment_success_rate)',
        'doctors(hosp_id, doctor_id, first_name, last_name, phone, email, address, begin_date, medical_specialty, diagnosis_id, med_degree, treatment_success_rate)',
        'diagnoses(diagnosis_id, disease_name, disease_type, malignant, disease_frequency, symptoms, treatment_success_rate)'
        
    
    ],
    'definition': [
        'Contains details about hospitals including unique ID, name, address, and how many doctors and nurses on staff, the treatment success rate, and what is the hospitals medical specialty.',
        'Contains details about patients including their unique ID, name, phone number, email address, home address, diagnosis, symptoms, the treatment success, and the date they were admitted into the hospital.',
        'Contains details about doctors including their unique ID, name, phone number, email address, home address, begin date working at the hospital, medical specialty and medical degree, treatment success rate, and the id of the diagnoses they specialize in.',
        'Contains details about diagnoses including their unique ID, name, type, whether is it malignant, the treatment success rate, the commonness of the disease and its symptoms.'
    ]
}

df = pd.DataFrame(data)
print(df)


  





                                               table  \
0  hospitals(hosp_id, name, phone, address, city,...   
1  patients(hosp_id,  patient_id, first_name, las...   
2  doctors(hosp_id, doctor_id, first_name, last_n...   
3  diagnoses(diagnosis_id, disease_name, disease_...   

                                          definition  
0  Contains details about hospitals including uni...  
1  Contains details about patients including thei...  
2  Contains details about doctors including their...  
3  Contains details about diagnoses including the...  


In [41]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [42]:
print(text_tables)

hospitals(hosp_id, name, phone, address, city, state, zip, doctors, nurses, medical_specialty): Contains details about hospitals including unique ID, name, address, and how many doctors and nurses on staff, and what is the hospitals medical specialty.
patients(hosp_id,  patient_id, first_name, last_name, admission_date, symptoms, diagnosis_id, phone, email, address): Contains details about patients including their unique ID, name, phone number, email address, home address, diagnosis, symptoms, and the date they were admitted into the hospital.
doctors(hosp_id, doctor_id, first_name, last_name, phone, email, address, begin_date, medical_specialty, diagnosis_id, med_degree): Contains details about doctors including their unique ID, name, phone number, email address, home address, begin date working at the hospital, medical specialty and medical degree, and the id of the diagnoses they specialize in.
diagnoses(diagnosis_id, disease_name, disease_type, malignant, disease_frequency, symptom

In [43]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [44]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="which doctors have worked at the hospital the longest?")

In [45]:
print(return_OAI(pqt1))

{
    "tables": ["doctors"]
}


In [46]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="which doctors specialize in cancer?")

In [47]:
print(return_OAI(pqt3))

```json
{
    "tables": ["doctors", "diagnoses"]
}
```


In [51]:
pqt4 = prompt_question_tables.format(tables=text_tables,
                                     question="which patients have multiple sclerosis and which doctors can treat them?")

In [52]:
print(return_OAI(pqt4))

{
    "tables": ["patients", "doctors", "diagnoses"]
}


In [55]:
pqt5 = prompt_question_tables.format(tables=text_tables,
                                     question="which hospitals have the most Harvard graduates working on their staff?")

In [56]:
print(return_OAI(pqt5))

```json
{
    "tables": ["hospitals", "doctors"]
}
```


In [58]:
pqt6 = prompt_question_tables.format(tables=text_tables,
                                     question="which hospitals have the most Harvard graduates employed?")

In [59]:
print(return_OAI(pqt6))

{
    "tables": ["hospitals", "doctors"]
}


In [60]:
pqt6 = prompt_question_tables.format(tables=text_tables,
                                     question="which doctors have the most success treating type 2 diabetes and which patients have they treated successfully?")

In [61]:
print(return_OAI(pqt6))

{
    "tables": ["doctors", "patients", "diagnoses"]
}


## Summary of Findings

The model is extremely useful for converting human questions into a table output. It seems that with clear definitions the model can infer what is meant when a question asks for information that is definition adjacent. For example, I asked the model "which hospitals have the most Harvard graduates working on their staff?". The definition for doctors includes a medical degree acknowledgement, but does not use the word "university" or "college" etc and it does not give examples of leanring institutions. However the model was able to infer from it's training that Harvard is a place where one can procure medical degrees, and therefore it knew to point me to the tables for hospitals and doctors, where the hospital would list which doctors it employs with a degree from Harvard.

I also chose to ask the same question but in different words to see if the model can infer from synonyms what is being asked. While the definitions uses the language "staff", I asked, "which hospitals have the most Harvard graduates employed?", using the word employed. The model was able to recognize the similarity of these words and produce the same table answer.

I also asked the model a more complex question, asking "which doctors have the most success treating type 2 diabetes and which patients have they treated successfully?", which did not confuse the model, as it was able to produce the tables: doctors, patients and diagnoses. 

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?